# Experiments in Modeling Issue Segmentation

## Load Libraries and Datasets

In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm
import altair as alt
alt.data_transformers.disable_max_rows()
from collections import deque
import warnings
warnings.filterwarnings('ignore')
from rich.console import Console
from rich.table import Table
import numpy as np
import scipy.stats as stats
from minineedle import needle, core
from sklearn.preprocessing import MinMaxScaler
console = Console()
import sys

sys.path.append("..")
from segmentation_scripts.utils import read_csv_file, get_data_directory_path

In [64]:
full_df = read_csv_file("../datasets/annotated_ht_ef_datasets/arab_observer_and_the_scribe/inu_32000013025079/inu_32000013025079_annotated_individual_tokens.csv")
console.print(f"Volume has this many tokens: {len(full_df)}")
console.print(f"Volume has this many issues: {full_df.start_issue.nunique()}")
console.print(f"Volume has this many pages: {full_df.page_number.nunique()}")
# Factorize the 'issue_number' column to create 'actual_issue_number'
full_df = full_df.sort_values(by=['page_number'])
full_df = full_df.rename(columns={'issue_number': 'original_issue_number'})
full_df['temp_issue_number'] = pd.factorize(full_df['original_issue_number'])[0]

# Expand count column
expanded_df = full_df.loc[full_df.index.repeat(full_df['count'])].reset_index(drop=True)
console.print(f"Expanded volume has this many tokens: {len(expanded_df)}")

# Calculate the number of tokens per page
tokens_per_page = expanded_df.groupby('page_number').size().reset_index(name='tokens_per_page')

# Merge the token counts back into the original DataFrame
expanded_df = expanded_df.merge(tokens_per_page, on='page_number', how='left')
missing_pages = full_df[~full_df.page_number.isin(expanded_df.page_number.unique())]
expanded_df = pd.concat([expanded_df, missing_pages], ignore_index=True)
expanded_df = expanded_df.reset_index(drop=True)


Volume has this many tokens: 228431

Volume has this many issues: 13

Volume has this many pages: 722

Expanded volume has this many tokens: 483990

In [65]:
def generate_table(df, table_title):
    # Create a Rich Table
    table = Table(title=table_title)
    columns = df.columns
    for column in columns:
        table.add_column(column.replace("_", " ").capitalize(), justify="center", style="cyan", no_wrap=True)    

    # Add rows to the table
    for _, row in df.iterrows():
        table.add_row(*[str(value) if pd.notna(value) else "" for value in row])

    # Print the table
    console.print(table)

### Clean Page Numbers

In [66]:
def filter_integers(token: str) -> bool:
	"""Check if the token is an integer."""
	return bool(re.match(r'^\d+$', token))

def calculate_digit_coverage(rows):
    """Calculate the number of digits in the given rows."""
    number_of_digits = rows['implied_zero'].notna().sum()
    return number_of_digits

def clean_digits(df: pd.DataFrame) -> pd.DataFrame:
	"""Clean and filter digit tokens in the DataFrame while retaining non-digit pages."""
	max_page = df.page_number.max()
	
	df['token'] = df['token'].astype(str)
	if 'volume_name' in df.columns:
		df['volume_number'] = df['volume_number'].fillna(0)
	
	# Identify pages with digit tokens
	subset_digits = df[df['token'].str.isdigit()].copy()
	possible_pages = subset_digits[subset_digits['token'].apply(filter_integers)].copy()
	# Use the smaller subset if possible_pages is smaller
	if len(possible_pages) < len(subset_digits):
		subset_digits = possible_pages
	non_digits_pages = df[(~df['token'].str.isdigit()) & (~df.page_number.isin(subset_digits.page_number))].copy()
	
	# just take the first page of non_digits_pages
	non_digits_pages = non_digits_pages.groupby('page_number').first().reset_index()
	print("non_digits_pages", non_digits_pages[non_digits_pages.page_number == 7])
	console.print(f"Number of digits in this volume: {len(subset_digits)}")
	console.print(f"Number of non-digit pages in this volume: {len(non_digits_pages)}")
	
	subset_digits['number'] = subset_digits['token'].astype(int, errors='ignore')
	max_possible_number = max_page + 25
	filtered_subset_digits = subset_digits[subset_digits['number'] < max_possible_number]
	non_filtered_subset_digits = subset_digits[(subset_digits['number'] >= max_possible_number) & (~subset_digits.page_number.isin(filtered_subset_digits.page_number))].groupby('page_number').first().reset_index()
	console.print(f"Number of digits in this volume after filtering for max page length: {len(filtered_subset_digits)}")
	console.print(f"Number of pages without digits in this volume after filtering for max page length: {len(non_filtered_subset_digits)}")
	
	# Calculate implied zero only for digit pages
	filtered_subset_digits['implied_zero'] = filtered_subset_digits['page_number'].astype(int) - filtered_subset_digits['number']
	
	final_subset_digits = filtered_subset_digits[filtered_subset_digits['implied_zero'] >= 0]
	console.print(f"Number of digits in this volume after filtering for max page length and implied zero: {len(final_subset_digits)}")
	remaining_missing_pages = df[(~df.page_number.isin(final_subset_digits.page_number)) & (~df.page_number.isin(non_digits_pages.page_number))].copy()
	remaining_missing_pages = remaining_missing_pages.groupby('page_number').first().reset_index()
	print("remaining_missing_pages", remaining_missing_pages[remaining_missing_pages.page_number == 7])
	console.print(f"Number of pages without digits in this volume after filtering for max page length and digit pages: {len(remaining_missing_pages)}")
	
	# Merge non-digit pages back into the DataFrame
	non_digits_pages['page_type'] = 'non_digit'
	remaining_missing_pages['page_type'] = 'negative_na_digit'
	final_subset_digits['page_type'] = 'digit'
	non_filtered_subset_digits['page_type'] = 'digit_too_large'
	full_df_with_digits = pd.concat([final_subset_digits, non_digits_pages, remaining_missing_pages, non_filtered_subset_digits]).sort_values(by=['page_number']).reset_index(drop=True)
	
	console.print(f"Number of pages after including non-digit pages: {full_df_with_digits.page_number.nunique()}")

	if full_df_with_digits.page_number.nunique() != df.page_number.nunique():
		added_pages = df[~df.page_number.isin(full_df_with_digits.page_number)].copy()
		added_pages = added_pages.groupby('page_number').first().reset_index()
		added_pages['page_type'] = 'added'
		full_df_with_digits = pd.concat([full_df_with_digits, added_pages]).sort_values(by=['page_number']).reset_index(drop=True)
		console.print(f"Number of pages after including added pages: {full_df_with_digits.page_number.nunique()}")

	# Calculate the number of digits per page
	tqdm.pandas(desc="Calculating digits per page")
	digits_per_page = full_df_with_digits.groupby('page_number').progress_apply(calculate_digit_coverage).reset_index(name='digits_per_page')
	full_df_with_digits = full_df_with_digits.merge(digits_per_page, on='page_number', how='left')
	
	return full_df_with_digits

# Example usage
subset_digits = clean_digits(expanded_df)
counts_per_annotated_issue = subset_digits.start_issue.value_counts().reset_index()
generate_table(counts_per_annotated_issue, "Counts per Annotated Issue")

non_digits_pages Empty DataFrame
Columns: [page_number, original_issue_number, dates, type_of_page, original_volumes, notes, start_issue, end_issue, link, htid, original_source, record_url, periodical_name, publication_type, id, metadata_schema_version, enumeration_chronology, type_of_resource, title, date_created, pub_date, language, access_profile, isbn, issn, lccn, oclc, page_count, feature_schema_version, access_rights, alternate_title, category, genre_ld, genre, contributor_ld, contributor, handle_url, source_institution_ld, source_institution, lcc, type, is_part_of, last_rights_update_date, pub_place_ld, pub_place, main_entity_of_page, publisher_ld, publisher, lowercase_periodical_name, publication_directory, volume_directory, section, token, pos, count, temp_issue_number, tokens_per_page]
Index: []

[0 rows x 57 columns]


Number of digits in this volume: 3925

Number of non-digit pages in this volume: 113

Number of digits in this volume after filtering for max page length: 2647

Number of pages without digits in this volume after filtering for max page length: 17

...

remaining_missing_pages Empty DataFrame
Columns: [page_number, original_issue_number, dates, type_of_page, original_volumes, notes, start_issue, end_issue, link, htid, original_source, record_url, periodical_name, publication_type, id, metadata_schema_version, enumeration_chronology, type_of_resource, title, date_created, pub_date, language, access_profile, isbn, issn, lccn, oclc, page_count, feature_schema_version, access_rights, alternate_title, category, genre_ld, genre, contributor_ld, contributor, handle_url, source_institution_ld, source_institution, lcc, type, is_part_of, last_rights_update_date, pub_place_ld, pub_place, main_entity_of_page, publisher_ld, publisher, lowercase_periodical_name, publication_directory, volume_directory, section, token, pos, count, temp_issue_number, tokens_per_page]
Index: []

[0 rows x 57 columns]


Number of pages without digits in this volume after filtering for max page length and digit pages: 20

Number of pages after including non-digit pages: 722

Calculating digits per page: 100%|██████████| 722/722 [00:00<00:00, 11078.55it/s]


 Counts per Annotated  
         Issue         
┏━━━━━━━━━━━━━┳━━━━━━━┓
┃ Start issue ┃ Count ┃
┡━━━━━━━━━━━━━╇━━━━━━━┩
│ 1965-02-07  │  214  │
│ 1966-01-03  │  213  │
│ 1965-02-21  │  206  │
│ 1965-01-10  │  205  │
│ 1965-02-28  │  199  │
│ 1965-01-17  │  195  │
│ 1965-03-21  │  193  │
│ 1966-01-31  │  192  │
│ 1965-03-28  │  192  │
│ 1965-02-14  │  191  │
│ 1965-03-07  │  190  │
│ 1966-01-24  │  187  │
│ 1965-03-14  │  171  │
└─────────────┴───────┘

In [67]:
distribution_df = subset_digits[['page_number', 'tokens_per_page', 'digits_per_page', 'start_issue']].drop_duplicates()

distribution_df['digit_ratio'] = distribution_df['digits_per_page'] / distribution_df['tokens_per_page']

# Calculate the mean digit ratio per issue
mean_digit_ratio_per_issue = distribution_df.groupby('start_issue')['digit_ratio'].mean().reset_index(name='mean_digit_ratio')
generate_table(mean_digit_ratio_per_issue, "Mean Digit Ratio per Issue")

      Mean Digit Ratio per Issue       
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Start issue ┃   Mean digit ratio    ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1965-01-10  │ 0.008234469636332948  │
│ 1965-01-17  │ 0.009039707926128971  │
│ 1965-02-07  │  0.01691876042169233  │
│ 1965-02-14  │ 0.008030563045521397  │
│ 1965-02-21  │ 0.011820424305336266  │
│ 1965-02-28  │ 0.0072910485997983755 │
│ 1965-03-07  │   0.006780551447329   │
│ 1965-03-14  │ 0.007865688776436003  │
│ 1965-03-21  │ 0.004881411374520723  │
│ 1965-03-28  │ 0.007860000809655938  │
│ 1966-01-03  │ 0.007860000180140393  │
│ 1966-01-24  │ 0.015630072140110852  │
│ 1966-01-31  │ 0.011709370973706022  │
└─────────────┴───────────────────────┘

In [68]:
alt.Chart(distribution_df).mark_circle().encode(
	x='page_number',
	y='digit_ratio',
	color='start_issue:N',
	tooltip=['page_number', 'digit_ratio', 'start_issue', 'tokens_per_page', 'digits_per_page']
).properties(
	width=400,
	height=200
)

alt.Chart(...)

### Detect Likely First Issue

In [84]:
def run_global_sequence_alignment(window: list, target_sequence: list, placeholder: int = -1) -> tuple:
    """Apply global sequence alignment on the implied zero values within a window using minineedle, with placeholders."""
    observed_sequence = [int(p[1]) if pd.notna(p[1]) else placeholder for p in window]
    
    # Check for valid entries in the observed sequence
    if all(val == placeholder for val in observed_sequence):
        return 0, [], []

    # Create Needleman-Wunsch global alignment instance
    alignment = needle.NeedlemanWunsch(observed_sequence, target_sequence)
    alignment.change_matrix(core.ScoreMatrix(match=4, miss=-0.5, gap=-1))

    try:
        # Run the alignment
        alignment.align()
        aligned_observed, aligned_target = alignment.get_aligned_sequences(core.AlignmentFormat.list)
        alignment_score = alignment.get_score()
        return alignment_score, aligned_observed, aligned_target

    except ZeroDivisionError:
        return 0, [], []

def sequence_alignment_issue_detection_global(df: pd.DataFrame, threshold_sizes: list, placeholder: int = -1) -> pd.DataFrame:
    """Detect issue boundaries using global sequence alignment."""
    df['page_number'] = df['page_number'].astype(int)
    df = df.sort_values(by=['page_number', 'implied_zero'])

    all_boundaries = []

    for threshold_size in tqdm(range(threshold_sizes[0], threshold_sizes[1]), desc="Threshold Sizes"):
        target_sequence = list(range(1, threshold_size + 1))  # Generate the target sequence
        first_page_number = df.page_number.min()
        final_page_number = df[df.page_number == first_page_number + threshold_size].page_number.max()
        
        for additional_page in range(5):
            current_first_page_number = first_page_number + additional_page
            selected_rows = df[(df.page_number <= final_page_number) & (df.page_number >= current_first_page_number)]
            potential_sequence = list(zip(selected_rows['page_number'], selected_rows['implied_zero']))
            
            # Run sequence alignment with placeholders
            alignment_score, aligned_observed, aligned_target = run_global_sequence_alignment(potential_sequence, target_sequence, placeholder=placeholder)
            
            # console.print(f"Alignment Score: {alignment_score}, Threshold Size: {threshold_size}, Start Page: {current_first_page_number}, End Page: {final_page_number}, length of target sequence: {len(target_sequence)}, length of potential sequence: {len(potential_sequence)}")
            # console.print(f"{alignment_score >= (0.6 * threshold_size)}, {alignment_score}, {0.6 * threshold_size}")
            # Analyze the alignment score
            if alignment_score > 0:  # Adjust this threshold as needed
                # console.print(f"Alignment Score: {alignment_score}, Threshold Size: {threshold_size}, Start Page: {current_first_page_number}, End Page: {final_page_number}, length of target sequence: {len(target_sequence)}, length of potential sequence: {len(potential_sequence)}")
                all_boundaries.append((alignment_score, aligned_observed, aligned_target, threshold_size, current_first_page_number, final_page_number))

    boundaries_df = pd.DataFrame(all_boundaries, columns=['alignment_score', 'aligned_observed', 'aligned_target', 'threshold_size', 'start_page', 'end_page'])
    return boundaries_df

dedup_subset_digits = subset_digits.drop_duplicates()
# Example usage
sequence_alignment_likely_first_issue_boundaries_df = sequence_alignment_issue_detection_global(dedup_subset_digits, threshold_sizes=[10, 200], placeholder=-1)

Threshold Sizes: 100%|██████████| 190/190 [00:44<00:00,  4.32it/s]


In [101]:
seventy_five_threshold = sequence_alignment_likely_first_issue_boundaries_df['alignment_score'].quantile(0.75)
top_ten_boundaries = sequence_alignment_likely_first_issue_boundaries_df[sequence_alignment_likely_first_issue_boundaries_df.alignment_score > seventy_five_threshold].sort_values(by=['alignment_score', 'start_page'], ascending=[False, True])
generate_table(top_ten_boundaries[['alignment_score', 'threshold_size', 'start_page', 'end_page']], "Top Ten Likely First Issue Boundaries")

           Top Ten Likely First Issue Boundaries            
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Alignment score ┃ Threshold size ┃ Start page ┃ End page ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│      29.5       │      59.0      │    11.0    │   66.0   │
│      28.5       │      59.0      │    10.0    │   66.0   │
│      27.0       │      58.0      │    11.0    │   65.0   │
│      26.5       │      59.0      │    8.0     │   66.0   │
│      26.5       │      59.0      │    9.0     │   66.0   │
│      26.0       │      58.0      │    10.0    │   65.0   │
│      24.5       │      59.0      │    7.0     │   66.0   │
│      24.5       │      60.0      │    11.0    │   67.0   │
│      24.0       │      58.0      │    8.0     │   65.0   │
│      24.0       │      58.0      │    9.0     │   65.0   │
│      23.5       │      60.0      │    10.0    │   67.0   │
│      23.5       │      56.0      │    11.0    │   63.0   │
│      23.0       │      57.0      │    11.0    │   64.0   │
│      22.5       │      56.0      │    10.0    │   63.0   │
│      22.0       │      58.0      │    7.0     │   65.0   │
│      22.0       │      57.0      │    10.0    │   64.0   │
│      22.0       │      61.0      │    11.0    │   68.0   │
│      21.5       │      60.0      │    8.0     │   67.0   │
│      21.5       │      60.0      │    9.0     │   67.0   │
│      21.5       │      63.0      │    11.0    │   70.0   │
│      21.0       │      61.0      │    10.0    │   68.0   │
│      20.5       │      56.0      │    8.0     │   63.0   │
│      20.5       │      56.0      │    9.0     │   63.0   │
│      20.5       │      63.0      │    10.0    │   70.0   │
│      20.5       │      65.0      │    11.0    │   72.0   │
│      20.0       │      57.0      │    8.0     │   64.0   │
│      20.0       │      57.0      │    9.0     │   64.0   │
│      19.5       │      60.0      │    7.0     │   67.0   │
│      19.5       │      65.0      │    10.0    │   72.0   │
│      19.5       │      55.0      │    11.0    │   62.0   │
│      19.5       │      62.0      │    11.0    │   69.0   │
│      19.0       │      61.0      │    8.0     │   68.0   │
│      19.0       │      61.0      │    9.0     │   68.0   │
│      19.0       │      64.0      │    11.0    │   71.0   │
│      18.5       │      56.0      │    7.0     │   63.0   │
│      18.5       │      63.0      │    8.0     │   70.0   │
│      18.5       │      63.0      │    9.0     │   70.0   │
│      18.5       │      55.0      │    10.0    │   62.0   │
│      18.5       │      62.0      │    10.0    │   69.0   │
│      18.5       │      53.0      │    11.0    │   60.0   │
│      18.0       │      57.0      │    7.0     │   64.0   │
│      18.0       │      64.0      │    10.0    │   71.0   │
│      17.5       │      65.0      │    8.0     │   72.0   │
│      17.5       │      65.0      │    9.0     │   72.0   │
│      17.5       │      53.0      │    10.0    │   60.0   │
│      17.5       │      32.0      │    11.0    │   39.0   │
│      17.5       │      44.0      │    11.0    │   51.0   │
│      17.5       │      50.0      │    11.0    │   57.0   │
│      17.5       │      54.0      │    11.0    │   61.0   │
│      17.5       │      66.0      │    11.0    │   73.0   │
│      17.0       │      61.0      │    7.0     │   68.0   │
│      17.0       │      48.0      │    11.0    │   55.0   │
│      17.0       │      69.0      │    11.0    │   76.0   │
│      16.5       │      63.0      │    7.0     │   70.0   │
│      16.5       │      55.0      │    8.0     │   62.0   │
│      16.5       │      62.0      │    8.0     │   69.0   │
│      16.5       │      55.0      │    9.0     │   62.0   │
│      16.5       │      62.0      │    9.0     │   69.0   │
│      16.5       │      32.0      │    10.0    │   39.0   │
│      16.5       │      44.0      │    10.0    │   51.0   │
│      16.5       │      50.0      │    10.0    │   57.0   │
│      16.5       │      54.0      

In [102]:
def select_likely_first_issue(df: pd.DataFrame, mean_threshold: float) -> pd.Series:
    """Select the most likely first issue based on weighted scores."""
    # Calculate frequency of start_page, end_page, and threshold_size
    start_page_freq = df['start_page'].value_counts(normalize=True).to_dict()
    end_page_freq = df['end_page'].value_counts(normalize=True).to_dict()
    # Define weights
    alpha, beta, gamma, delta = 0.4, 0.2, 0.2, 0.2

    # Add a column for weighted score
    def calculate_weighted_score(row):
        alignment_score = row['alignment_score']
        start_page_score = start_page_freq.get(row['start_page'], 0)
        end_page_score = end_page_freq.get(row['end_page'], 0)
        threshold_diff = abs(row['threshold_size'] - mean_threshold)
        threshold_score = 1 - (threshold_diff / mean_threshold)

        return (alpha * alignment_score +
                beta * start_page_score +
                gamma * end_page_score +
                delta * threshold_score)

    df['weighted_score'] = df.apply(calculate_weighted_score, axis=1)

    # Select the candidate with the highest weighted score
    best_candidate = df.sort_values(by='weighted_score', ascending=False).iloc[0]

    return best_candidate

def calculate_confidence_interval(df: pd.DataFrame, column: str, confidence: float = 0.95) -> tuple:
    """Calculate the confidence interval for a given column."""
    mean_val = df[column].mean()
    std_dev = df[column].std()
    n = len(df)

    # Calculate standard error
    standard_error = std_dev / np.sqrt(n)

    # Calculate confidence interval
    z_score = stats.norm.ppf((1 + confidence) / 2)
    margin_of_error = z_score * standard_error

    lower_bound = mean_val - margin_of_error
    upper_bound = mean_val + margin_of_error

    return mean_val, lower_bound, upper_bound, margin_of_error


# Calculate the mean of threshold sizes
mean_threshold = top_ten_boundaries['threshold_size'].mean()

# Apply the selection function to the top ten boundaries
best_first_issue = select_likely_first_issue(top_ten_boundaries, mean_threshold)
best_first_issue_df = pd.DataFrame([best_first_issue]).reset_index(drop=True)
generate_table(best_first_issue_df[['alignment_score', 'threshold_size', 'start_page', 'end_page']], "Best First Issue Candidate")

# Calculate confidence intervals for threshold_size and alignment_score
mean_threshold, lower_threshold, upper_threshold, margin_error_threshold = calculate_confidence_interval(top_ten_boundaries, 'threshold_size')
mean_score, lower_score, upper_score, margin_error_score = calculate_confidence_interval(top_ten_boundaries, 'alignment_score')

console.print(f"Threshold Size: Mean = {mean_threshold}, CI = ({lower_threshold}, {upper_threshold}), Margin of Error = {margin_error_threshold}", style="bold")
console.print(f"Alignment Score: Mean = {mean_score}, CI = ({lower_score}, {upper_score}), Margin of Error = {margin_error_score}", style="bold")


                 Best First Issue Candidate                 
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Alignment score ┃ Threshold size ┃ Start page ┃ End page ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│      29.5       │      59.0      │    11.0    │   66.0   │
└─────────────────┴────────────────┴────────────┴──────────┘

Threshold Size: Mean = 55.10752688172043, CI = (53.01602925175911, 57.199024511681756), Margin of Error = 
2.091497629961324

Alignment Score: Mean = 18.446236559139784, CI = (17.705607867492912, 19.186865250786656), Margin of Error = 
0.7406286916468718

In [103]:
def probabilistic_first_issue_detection(df: pd.DataFrame, threshold_sizes: list, window_size: int = 5, score_threshold: float = 0.5) -> pd.DataFrame:
	"""Identify the likely first issue length using probabilistic detection."""
	df['page_number'] = df['page_number'].astype(int)
	df['implied_zero'] = df['implied_zero'].astype(int, errors='ignore')
	df = df.sort_values(by=['page_number'])

	all_boundaries = []

	for threshold_size in tqdm(range(threshold_sizes[0], threshold_sizes[1]), desc="Threshold Sizes"):
		first_page_number = df.page_number.min()
		final_page_number = df[df.page_number == first_page_number + threshold_size].page_number.max()

		# Vary the start page within a defined range (similar to Needleman-Wunsch approach)
		for additional_page in range(5):
			current_first_page_number = first_page_number + additional_page
			selected_rows = df[(df.page_number <= final_page_number) & (df.page_number >= current_first_page_number)]

			sliding_window = deque(maxlen=window_size)
			cumulative_score = 0

			# Iterate through the observed sequence in the current window
			for _, row in selected_rows.iterrows():
				page_number = row['page_number']
				implied_zero = row['implied_zero'] if pd.notna(row['implied_zero']) else None
				section_weight = 0.2 if row['section'] != "body" else 0

				# Add to the sliding window
				if implied_zero is not None:
					sliding_window.append((page_number, implied_zero, section_weight))
				else:
					sliding_window.append((page_number, None, 0))

				# Calculate scores once the window is full
				# Check if the window is full and contains valid implied_zero values
				non_none_values = [p for p in sliding_window if p[1] is not None]
				if len(sliding_window) == window_size and non_none_values:
					page_range = max(p[0] for p in sliding_window if p[1] is not None) - min(p[0] for p in sliding_window if p[1] is not None)
					implied_zero_diff = max(p[1] for p in sliding_window if p[1] is not None) - min(p[1] for p in sliding_window if p[1] is not None)

					score = 0
					if page_range > threshold_size:
						score += 0.7

					if implied_zero_diff > threshold_size:
						score += 0.5

					non_digit_count = sum(1 for p in sliding_window if p[1] is None)
					if non_digit_count > 0:
						score += 0.25 * (non_digit_count / window_size)

					section_weight = sum(p[2] for p in sliding_window)
					if section_weight > 0:
						score += 0.2 * section_weight

					# Accumulate scores and evaluate threshold
					cumulative_score += score
					if cumulative_score >= score_threshold:
						
						all_boundaries.append((
							cumulative_score, sliding_window, threshold_size,
							current_first_page_number, final_page_number
						))
						cumulative_score = 0  # Reset cumulative score

	boundaries_df = pd.DataFrame(all_boundaries, columns=[
		'cumulative_score', 'sliding_window', 'threshold_size', 'start_page', 'end_page'
	])
	return boundaries_df


# Example usage
sliding_window_prob_first_issue_df = probabilistic_first_issue_detection(subset_digits, threshold_sizes=[round(lower_threshold), round(upper_threshold)], window_size=5, score_threshold=0.5)

# Analyze the top candidates for the first issue
seventy_five_threshold = sliding_window_prob_first_issue_df.describe()[['cumulative_score']].T['75%'].values[0]
top_prob_candidates = sliding_window_prob_first_issue_df[sliding_window_prob_first_issue_df.cumulative_score > seventy_five_threshold].sort_values(by=['cumulative_score', 'start_page'], ascending=[False, True])
generate_table(top_prob_candidates[['cumulative_score', 'threshold_size', 'start_page', 'end_page']], "Top Ten Probabilistic First Issue Candidates")

Threshold Sizes: 100%|██████████| 4/4 [00:00<00:00, 21.52it/s]


         Top Ten Probabilistic First Issue Candidates          
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃  Cumulative score  ┃ Threshold size ┃ Start page ┃ End page ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 0.6300000000000001 │      53.0      │    8.0     │   60.0   │
│ 0.6300000000000001 │      54.0      │    8.0     │   61.0   │
│ 0.6300000000000001 │      55.0      │    8.0     │   62.0   │
│ 0.6300000000000001 │      56.0      │    8.0     │   63.0   │
│ 0.6300000000000001 │      53.0      │    9.0     │   60.0   │
│ 0.6300000000000001 │      54.0      │    9.0     │   61.0   │
│ 0.6300000000000001 │      55.0      │    9.0     │   62.0   │
│ 0.6300000000000001 │      56.0      │    9.0     │   63.0   │
│ 0.6100000000000001 │      53.0      │    7.0     │   60.0   │
│ 0.6100000000000001 │      54.0      │    7.0     │   61.0   │
│ 0.6100000000000001 │      55.0      │    7.0     │   62.0   │
│ 0.6100000000000001 │      56.0      │    7.0     │   63.0   │
│ 0.6000000000000001 │      53.0      │    7.0     │   60.0   │
│ 0.6000000000000001 │      54.0      │    7.0     │   61.0   │
│ 0.6000000000000001 │      55.0      │    7.0     │   62.0   │
│ 0.6000000000000001 │      56.0      │    7.0     │   63.0   │
│ 0.5800000000000001 │      53.0      │    7.0     │   60.0   │
│ 0.5800000000000001 │      54.0      │    7.0     │   61.0   │
│ 0.5800000000000001 │      55.0      │    7.0     │   62.0   │
│ 0.5800000000000001 │      56.0      │    7.0     │   63.0   │
│ 0.5800000000000001 │      53.0      │    8.0     │   60.0   │
│ 0.5800000000000001 │      54.0      │    8.0     │   61.0   │
│ 0.5800000000000001 │      55.0      │    8.0     │   62.0   │
│ 0.5800000000000001 │      56.0      │    8.0     │   63.0   │
│ 0.5800000000000001 │      53.0      │    9.0     │   60.0   │
│ 0.5800000000000001 │      54.0      │    9.0     │   61.0   │
│ 0.5800000000000001 │      55.0      │    9.0     │   62.0   │
│ 0.5800000000000001 │      56.0      │    9.0     │   63.0   │
│ 0.5800000000000001 │      53.0      │    10.0    │   60.0   │
│ 0.5800000000000001 │      54.0      │    10.0    │   61.0   │
│ 0.5800000000000001 │      55.0      │    10.0    │   62.0   │
│ 0.5800000000000001 │      56.0      │    10.0    │   63.0   │
│ 0.5800000000000001 │      53.0      │    11.0    │   60.0   │
│ 0.5800000000000001 │      54.0      │    11.0    │   61.0   │
│ 0.5800000000000001 │      55.0      │    11.0    │   62.0   │
│ 0.5800000000000001 │      56.0      │    11.0    │   63.0   │
│ 0.5700000000000003 │      53.0      │    10.0    │   60.0   │
│ 0.5700000000000003 │      54.0      │    10.0    │   61.0   │
│ 0.5700000000000003 │      55.0      │    10.0    │   62.0   │
│ 0.5700000000000003 │      56.0      │    10.0    │   63.0   │
│ 0.5700000000000001 │      53.0      │    7.0     │   60.0   │
│ 0.5700000000000001 │      54.0      │    7.0     │   61.0   │
│ 0.5700000000000001 │      55.0      │    7.0     │   62.0   │
│ 0.5700000000000001 │      56.0      │    7.0     │   63.0   │
│ 0.5700000000000001 │      53.0      │    8.0     │   60.0   │
│ 0.5700000000000001 │      54.0      │    8.0     │   61.0   │
│ 0.5700000000000001 │      55.0      │    8.0     │   62.0   │
│ 0.5700000000000001 │      56.0      │    8.0     │   63.0   │
│ 0.5700000000000001 │      53.0      │    9.0     │   60.0   │
│ 0.5700000000000001 │      54.0      │    9.0     │   61.0   │
│ 0.5700000000000001 │      55.0      │    9.0     │   62.0   │
│ 0.5700000000000001 │      56.0      │    9.0     │   63.0   │
│ 0.5700000000000001 │      53.0      │    10.0    │   60.0   │
│ 0.5700000000000001 │      54.0      │    10.0    │   61.0   │
│ 0.5700000000000001 │      55.0      │    10.0    │   62.0   │
│ 0.5700000000000001 │      56.0      │    10.0    │   63.0   │
│ 0.5700000000000001 │      53.0      │    11.0    │   60.0   │
│ 0.5700000000000001 │      54.0      │    11.0    │   61.0   │
│ 0.5700000000000001 │      55.0

In [104]:
# Adjusted Raw Scores Initialization
def initialize_raw_scores(df, max_threshold):
	max_page = df['page_number'].max()
	if pd.isna(max_page):
		return np.zeros((0, 0), dtype=int)
	raw_scores = np.zeros((max_page + 1, max_threshold + 1), dtype=int)

	for _, row in df.iterrows():
		page = int(row['page_number'])
		number = int(row['implied_zero']) if row['page_type'] == 'digit' else 0
		
		if 0 <= page <= max_page and 0 <= number <= max_threshold:
			raw_scores[page, number] += 1

	return raw_scores

# Modified Prefix Sum Calculation for First Issue
def prefix_sums_first_issue(raw_scores, threshold_range, start_pages, updown=0.5, diag=0.25, otherwise=0.01, points=1.0):
	nrows, ncols = raw_scores.shape
	max_score_data = []

	# Iterate over threshold sizes
	for threshold_size in threshold_range:
		# Iterate over start pages extracted from the DataFrame
		for start_page in start_pages:
			end_page = start_page + threshold_size - 1

			# Ensure the end page doesn't exceed the matrix bounds
			if end_page >= nrows:
				continue
			
			# Initialize prefix sum matrix for the current configuration
			current_scores = raw_scores.copy()

			# Apply prefix sums within the current window
			for i in range(start_page, end_page + 1):
				for j in range(ncols):
					cell = otherwise + points * raw_scores[i, j]
					choices = []

					if j > 0:
						choices.append(current_scores[i, j-1] * updown)
					if i > start_page:
						choices.append(current_scores[i-1, j] * updown)
						if j > 0:
							choices.append(current_scores[i-1, j-1] * diag)

					cell += max(choices, default=0)
					current_scores[i, j] = cell

			# Calculate the total score for this configuration
			total_score = current_scores[start_page:end_page + 1, :].sum()

			# Collect the configuration and its total score
			max_score_data.append((total_score, threshold_size, start_page, end_page))

	return max_score_data

# Analyze Prefix Sum Results for First Issue
def detect_first_issue_prefix_sum(df, threshold_range=[10, 50], updown=0.5, diag=0.25, otherwise=0.01, points=1.0):
	max_score_data = []

	for threshold_size in tqdm(range(threshold_range[0], threshold_range[1]), desc="Threshold Sizes"):
		first_page_number = df.page_number.min()
		final_page_number = df[df.page_number == first_page_number + threshold_size].page_number.max()

		# Vary the start page within a defined range (similar to Needleman-Wunsch approach)
		for additional_page in range(5):
			current_first_page_number = first_page_number + additional_page
			selected_rows = df[(df.page_number <= final_page_number) & (df.page_number >= current_first_page_number)]
			# Initialize raw scores matrix
			raw_scores = initialize_raw_scores(selected_rows, max_threshold=threshold_range[1])

			# Extract unique page numbers to use as start pages
			start_pages = selected_rows['page_number'].unique()

			# Run prefix sums across different thresholds and start pages
			max_score_data.extend(prefix_sums_first_issue(
				raw_scores,
				range(threshold_range[0], threshold_range[1]),
				start_pages,
				updown,
				diag,
				otherwise,
				points
			))

	# Convert to DataFrame
	results_df = pd.DataFrame(max_score_data, columns=['total_score', 'threshold_size', 'start_page', 'end_page'])
	results_df['threshold_size'] = results_df.end_page - results_df.start_page 
	best_candidate = results_df.sort_values(by='total_score', ascending=False).head(1)
	
	return best_candidate, results_df

# Example usage with subset_digits DataFrame
best_first_issue, prefix_all_candidates_df = detect_first_issue_prefix_sum(subset_digits, threshold_range=[round(lower_threshold), round(upper_threshold)])

generate_table(best_first_issue[['total_score', 'threshold_size', 'start_page', 'end_page']], "Best First Issue Candidate")

Threshold Sizes: 100%|██████████| 4/4 [00:00<00:00,  4.08it/s]


               Best First Issue Candidate               
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Total score ┃ Threshold size ┃ Start page ┃ End page ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│     314     │       55       │     7      │    62    │
└─────────────┴────────────────┴────────────┴──────────┘

In [105]:
# Analyze the top candidates for the first issue
seventy_five_threshold = prefix_all_candidates_df.describe()[['total_score']].T['75%'].values[0]
top_prob_candidates = prefix_all_candidates_df[prefix_all_candidates_df.total_score > seventy_five_threshold].sort_values(by=['total_score', 'start_page'], ascending=[False, True])
generate_table(top_prob_candidates[['total_score', 'threshold_size', 'start_page', 'end_page']], "Top Prefix Sum First Issue Candidates")

         Top Prefix Sum First Issue Candidates          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Total score ┃ Threshold size ┃ Start page ┃ End page ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│     314     │       55       │     7      │    62    │
│     314     │       55       │     7      │    62    │
│     313     │       54       │     9      │    63    │
│     313     │       54       │     9      │    63    │
│     313     │       54       │     9      │    63    │
│     312     │       53       │     9      │    62    │
│     312     │       53       │     9      │    62    │
│     312     │       53       │     9      │    62    │
│     312     │       53       │     9      │    62    │
│     312     │       53       │     9      │    62    │
│     312     │       53       │     9      │    62    │
└─────────────┴────────────────┴────────────┴──────────┘

In [106]:
top_issues = prefix_all_candidates_df[['total_score', 'threshold_size', 'start_page', 'end_page']].merge(sequence_alignment_likely_first_issue_boundaries_df[['alignment_score', 
        'threshold_size', 'start_page', 'end_page']], on=['threshold_size', 'start_page', 'end_page'], how='outer').sort_values(by=['total_score', 'alignment_score'], ascending=[False, False])

top_issues_df = top_issues.merge(sliding_window_prob_first_issue_df[['cumulative_score','threshold_size', 'start_page', 'end_page']], on=['threshold_size', 'start_page', 'end_page'], how='inner').sort_values(by=['total_score', 'cumulative_score'], ascending=[False, False])

top_issues_df = top_issues_df.drop_duplicates()
generate_table(top_issues_df[['threshold_size', 'start_page', 'end_page', 'alignment_score', 'total_score', 'cumulative_score']], "Top First Issue Candidates")

                                  Top First Issue Candidates                                   
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Threshold size ┃ Start page ┃ End page ┃ Alignment score ┃ Total score ┃  Cumulative score  ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.6100000000000001 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.6000000000000001 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5800000000000001 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5700000000000001 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.56        │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.55        │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5499999999999999 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.54        │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5300000000000002 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.53        │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5200000000000001 │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.52        │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.6100000000000001 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.6000000000000001 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5800000000000001 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5700000000000001 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.56        │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.55        │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5499999999999999 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.54        │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5300000000000002 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.53        │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5200000000000001 │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.52        │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.6100000000000001 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.6000000000000001 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5800000000000001 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5700000000000001 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.56        │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.55        │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5499999999999999 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.54        │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5300000000000002 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.53        │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5200000000000001 │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.52        │
│      56.0      │    8.0     │   63.0   │      20.5       │             │ 0.6300000000000001 │
│      56.0      │    9.0     │   63.0   │      20.5       │    

In [107]:
weights = {
    'total_score': 0.4,
    'alignment_score': 0.4,
    'cumulative_score': 0.2
}

top_issues_df['composite_score'] = (
    weights['total_score'] * top_issues_df['total_score'] +
    weights['alignment_score'] * top_issues_df['alignment_score'] +
    weights['cumulative_score'] * top_issues_df['cumulative_score']
)

top_issues_df = top_issues_df.sort_values(by='composite_score', ascending=False)
generate_table(top_issues_df[['threshold_size', 'start_page', 'end_page', 'alignment_score', 'total_score', 'cumulative_score', 'composite_score']], "Top First Issue Candidates with Composite Score")

                                  Top First Issue Candidates with Composite Score                                  
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Threshold size ┃ Start page ┃ End page ┃ Alignment score ┃ Total score ┃  Cumulative score  ┃  Composite score  ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.6100000000000001 │ 131.522000000000… │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.6000000000000001 │      131.52       │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5800000000000001 │ 131.516000000000… │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5700000000000001 │      131.514      │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.56        │      131.512      │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.55        │ 131.510000000000… │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5499999999999999 │ 131.510000000000… │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.54        │      131.508      │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.53        │      131.506      │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5300000000000002 │      131.506      │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │ 0.5200000000000001 │ 131.504000000000… │
│      55.0      │    7.0     │   62.0   │      14.5       │    314.0    │        0.52        │ 131.504000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.6100000000000001 │ 129.522000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.6000000000000001 │      129.52       │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5800000000000001 │ 129.516000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5700000000000001 │      129.514      │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.56        │      129.512      │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5499999999999999 │ 129.510000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.55        │ 129.510000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.54        │      129.508      │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5300000000000002 │      129.506      │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.53        │      129.506      │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │ 0.5200000000000001 │ 129.504000000000… │
│      54.0      │    7.0     │   61.0   │      12.5       │    311.0    │        0.52        │ 129.504000000000… │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.6100000000000001 │ 127.922000000000… │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.6000000000000001 │ 127.920000000000… │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5800000000000001 │ 127.916000000000… │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │ 0.5700000000000001 │ 127.914000000000… │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.56        │      127.912      │
│      53.0      │    7.0     │   60.0   │      13.5       │    306.0    │        0.55        │ 127.910000000000… │
│      53.0      │    7.0     │   60.0   │      13.5    

In [108]:


def calculate_combined_score(df, total_weight=0.4, alignment_weight=0.3, cumulative_weight=0.3):
    """Calculate a combined score based on total_score, alignment_score, and cumulative_score."""
    # Normalize the scores using Min-Max Scaling
    scaler = MinMaxScaler()

    df[['norm_total_score', 'norm_alignment_score', 'norm_cumulative_score']] = scaler.fit_transform(
        df[['total_score', 'alignment_score', 'cumulative_score']]
    )

    # Calculate the combined score as a weighted sum
    df['combined_score'] = (
        df['norm_total_score'] * total_weight +
        df['norm_alignment_score'] * alignment_weight +
        df['norm_cumulative_score'] * cumulative_weight
    )

    return df

# Apply the function to your DataFrame
top_issues_df = calculate_combined_score(top_issues_df)

In [109]:
alt.Chart(top_issues_df).mark_circle().encode(
	x='composite_score',
	y='combined_score',
	color='threshold_size:N',
	tooltip=['threshold_size', 'start_page', 'end_page', 'alignment_score', 'total_score', 'cumulative_score', 'composite_score']
)

alt.Chart(...)

In [114]:
annotated_df = full_df[['page_number', 'start_issue', 'end_issue', 'type_of_page']].drop_duplicates()

# Group by 'start_issue' and aggregate
grouped_df = annotated_df.groupby('start_issue').agg(
    first_page=('page_number', 'min'),
    last_page=('page_number', 'max'),
    number_of_pages=('page_number', 'count')
).reset_index()
grouped_df = grouped_df.sort_values(by='first_page')

In [113]:
grouped_df.sor

,start_issue,first_page,last_page,number_of_pages
0,1965-01-10,63,118,56
1,1965-01-17,119,176,58
2,1965-02-07,289,344,56
3,1965-02-14,345,400,56
4,1965-02-21,401,455,55
5,1965-02-28,457,512,55
6,1965-03-07,513,567,55
7,1965-03-14,569,624,56
8,1965-03-21,625,680,56
9,1965-03-28,681,736,55


In [115]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def generate_issue_binary(start_page, end_page, total_pages):
	"""
	Convert the issue boundaries into a binary format indicating issue presence.
	"""
	issue_binary = np.zeros(total_pages, dtype=int)
	issue_binary[int(start_page):int(end_page) + 1] = 1
	return issue_binary

def calculate_first_issue_accuracy(top_issues_df, grouped_df, total_pages):
	"""
	Calculate accuracy, precision, recall, and F1-score for first issue detection.
	"""
	# Extract the first row from top_issues_df as the predicted first issue
	metrics_df = top_issues_df.copy()
	for index, row in tqdm(top_issues_df.iterrows(), total=top_issues_df.shape[0], desc="Calculating Metrics"):
		# Extract the predicted first issue boundaries
		predicted_start_page = int(row['start_page'])
		predicted_end_page = int(row['end_page'])
		# Extract the actual first issue boundaries from grouped_df
		actual_first_issue = grouped_df.iloc[0]
		actual_start_page = int(actual_first_issue['first_page'])
		actual_end_page = int(actual_first_issue['last_page'])
		actual_issue_length = int(actual_first_issue['number_of_pages'])

		# Convert predicted and actual issues to binary format
		predicted_issues_binary = generate_issue_binary(predicted_start_page, predicted_end_page, total_pages)
		actual_issues_binary = generate_issue_binary(actual_start_page, actual_end_page, total_pages)

		# Calculate accuracy, precision, recall, and F1-score
		accuracy = accuracy_score(actual_issues_binary, predicted_issues_binary)
		precision = precision_score(actual_issues_binary, predicted_issues_binary)
		recall = recall_score(actual_issues_binary, predicted_issues_binary)
		f1 = f1_score(actual_issues_binary, predicted_issues_binary)

		metrics_df.loc[index, 'accuracy'] = accuracy
		metrics_df.loc[index, 'precision'] = precision
		metrics_df.loc[index, 'recall'] = recall
		metrics_df.loc[index, 'f1'] = f1
		metrics_df.loc[index, 'actual_start_page'] = actual_start_page
		metrics_df.loc[index, 'actual_end_page'] = actual_end_page
		metrics_df.loc[index, 'actual_total_volume_pages'] = total_pages
		metrics_df.loc[index, 'actual_issue_length'] = actual_issue_length


	return metrics_df

# Define the total number of pages in the volume
total_pages = full_df['page_number'].max() + 1

# Calculate accuracy, precision, recall, and F1-score for the first issue detection
metrics_df = calculate_first_issue_accuracy(top_issues_df, grouped_df, total_pages)

Calculating Metrics: 100%|██████████| 200/200 [00:00<00:00, 407.95it/s]


In [116]:
metrics_df = metrics_df.sort_values(by='f1', ascending=False)
generate_table(metrics_df[['threshold_size', 'start_page', 'end_page', 'accuracy', 'precision', 'recall', 'f1']], "First Issue Detection Metrics")

                                           First Issue Detection Metrics                                           
┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Threshold s… ┃ Start p… ┃ End pa… ┃     Accuracy     ┃     Precision     ┃      Recall      ┃        F1         ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     55.0     │   7.0    │  62.0   │       1.0        │        1.0        │       1.0        │        1.0        │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     56.0     │   7.0    │  63.0   │ 0.9986431478968… │ 0.98245614035087… │       1.0        │ 0.99115044247787… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │        1.0        │ 0.9821428571428… │ 0.99099099099099… │
│     55.0     │   8.0    │  62.0   │ 0.9986431478968… │

## Explore Issue Segmentation Metrics

In [133]:
identified_issues_dfs = []
for directory, subdir, files in os.walk("../datasets/first_issue_metrics"):
	for file in files:
		if file.endswith(".csv"):
			df = read_csv_file(os.path.join(directory, file))
			identified_issues_dfs.append(df)

In [134]:
identified_issues_df = pd.concat(identified_issues_dfs, ignore_index=True)

In [135]:
identified_issues_df[0:1]

,total_score,threshold_size,start_page,end_page,alignment_score,cumulative_score,composite_score,norm_total_score,norm_alignment_score,norm_cumulative_score,combined_score,accuracy,precision,recall,f1,actual_start_page,actual_end_page,actual_total_volume_pages,actual_issue_length,annotated_file_path
0,535.0,69,3,72,37.0,NaN,NaN,0.0,0.0,NaN,NaN,0.386667,0.342857,1.0,0.510638,3.0,26.0,75.0,24.0,../datasets/annotated_ht_ef_datasets/liberator...


In [136]:
identified_issues_df.groupby(['annotated_file_path', 'actual_issue_length', 'actual_total_volume_pages'])['f1'].max().reset_index().sort_values(by='f1', ascending=False)

,annotated_file_path,actual_issue_length,actual_total_volume_pages,f1
0,../datasets/annotated_ht_ef_datasets/arab_obse...,52.0,305.0,1.000000
6,../datasets/annotated_ht_ef_datasets/arab_obse...,34.0,963.0,1.000000
20,../datasets/annotated_ht_ef_datasets/liberator...,32.0,390.0,1.000000
9,../datasets/annotated_ht_ef_datasets/arab_obse...,55.0,957.0,0.991150
3,../datasets/annotated_ht_ef_datasets/arab_obse...,55.0,737.0,0.982456
11,../datasets/annotated_ht_ef_datasets/arab_obse...,53.0,959.0,0.982143
2,../datasets/annotated_ht_ef_datasets/arab_obse...,54.0,741.0,0.956522
13,../datasets/annotated_ht_ef_datasets/arab_obse...,56.0,819.0,0.949153
22,../datasets/annotated_ht_ef_datasets/liberator...,24.0,51.0,0.923077
5,../datasets/annotated_ht_ef_datasets/arab_obse...,53.0,739.0,0.896000
